# Camera calibration

Notebook is based on OpenCV tutorial (https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html)


see also for visualization:

https://www.rerun.io/


online viewer:

https://www.rerun.io/viewer


In [ ]:
!wget -O samples.csv https://raw.githubusercontent.com/ant-nik/semares/master/data/stereo-camera-cyl/position.csv

In [ ]:
import cv2
import logging
import io
import numpy
import plotly.express as plte


logger = logging.getLogger(__name__)
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


def load_image(payload: any) -> any:
    np_image = numpy.frombuffer(payload, numpy.uint8)
    img = cv2.imdecode(np_image, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    return img


def get_chess_corners_from_image(
        image: any, corners_x: int, corners_y: int) -> tuple[any, any]:
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (corners_x, corners_y), None)

    # If found, add object points, image points (after refining them)
    if ret != True:
        logger.error("Error, grid (%d, %d) is not found in image",
                     corners_x, corners_y)
        return None, gray

    corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1,-1), criteria)
    result_image = cv2.drawChessboardCorners(image, (7,7), corners, True)
    return corners2, result_image


def undistort_image(image: numpy.ndarray, mtx: numpy.ndarray,
                    dist: numpy.ndarray, new_mtx: numpy.ndarray,
                    canvas_w: int, canvas_h: int) -> numpy.ndarray:
    mapx, mapy = cv2.initUndistortRectifyMap(mtx, dist, None, new_mtx,
                                             (canvas_w, canvas_h), 5)
    return cv2.remap(image, mapx, mapy, cv2.INTER_LINEAR)


In [ ]:
import pandas
import logging
import functools


logger = logging.getLogger(__name__)


def load_single_image(item: pandas.Series, url_template: str) -> numpy.ndarray:
    item_file = item["image"].lstrip(" ")
    url = url_template.format(item_file)
    response = requests.get(url)
    if response.status_code != 200:
        logger.error("Can't read image %s from url %s", item_file, url)
    return pandas.Series([load_image(response.content), url],
                         index=["image_data", "url"])


def load_images_and_meta(filename: str, base_url: str) -> pandas.DataFrame:
    dataframe = pandas.read_csv(filename, sep=',')
    orig_files = []
    for index, item in dataframe.iterrows():
        item_file = item["image"].lstrip(" ")
        url = base_url.format(item_file)
        response = requests.get(url)
        if response.status_code != 200:
            logger.error("Can't read image %s from url %s", item_file, url)
        # Draw and display the corners
        orig_image = load_image(response.content)
        orig_files.append({
            "orig_image": numpy.array(orig_image),
            "url": url
        })
    dataframe.assign(orig_image=orig_files)
    return dataframe

In [ ]:
import requests
import logging


logger = logging.getLogger(__name__)


def match_points(
        corners_x: numpy.ndarray,
        corners_y: numpy.ndarray,
        corners: numpy.ndarray,
) -> tuple[numpy.ndarray, numpy.ndarray]:
    objp = numpy.zeros((corners_x*corners_y, 3), numpy.float32)
    objp[:,:2] = numpy.mgrid[0:corners_x, 0:corners_y].T.reshape(-1, 2)
    return objp


def calculate_image_corners(
        row: pandas.Series, ignore_list: str, shapes: numpy.ndarray
) -> pandas.Series:
    if row["image"] in ignore_list:
        return pandas.Series(
            [
                None,
                None
            ], index=["corners", "chess_image"]
        )
    corners, image = get_chess_corners_from_image(
        image=row["image_data"],
        corners_x=row["corners_x"],
        corners_y=row["corners_y"]
    )

    img_shape = (row["image_data"].shape[1], row["image_data"].shape[0])
    #if shapes != img_shape:
    #    logger.error(
    #        "Image %s has specific shape %s that is not equals to previously shapes: %s",
    #        sample.name, str(img_shape), str(shapes)
    #        )
    #    return pandas.Series([None, None, None], index=["obj_points", "image_points"])

    if corners is None:
        objp = None
        logger.error("Can't find %d, %d conrenrs in %s",
                     row["corners_x"], row["corners_y"], row["image"])
    else:
        objp = match_points(corners_x=row["corners_x"],
                            corners_y=row["corners_y"],
                            corners=corners)

    return pandas.Series([
        corners,
        image,
        objp
    ], index=["corners", "chess_image", "object_points"])


In [ ]:
base_url = "https://raw.githubusercontent.com/ant-nik/semares/master/data/stereo-camera-cyl/{}"
ignore_list = [
    "image75_r.jpg", "image84_r.jpg",
    "image84_l.jpg", "image89_r.jpg",
    "image89_l.jpg", "image91_r.jpg",
    "image91_l.jpg"]

dataframe = pandas.read_csv("samples.csv", sep=',')
dataframe = dataframe.join(
    dataframe.apply(
        functools.partial(
            load_single_image,
            url_template=base_url),
        axis=1)
)
shapes = dataframe["image_data"].apply(lambda x: x.shape).unique()
if shapes.shape[0] != 1:
    logger.error("More than one unique image sizes were found: %s",
                 str(shapes))
shapes = (shapes[0][1], shapes[0][0])
shapes

In [ ]:
dataframe = dataframe.join(
    dataframe.apply(
        functools.partial(
            calculate_image_corners,
            ignore_list=ignore_list,
            shapes=None),
        axis=1)
)
dataframe.columns

In [ ]:
filtered = dataframe[~dataframe["image"].isin(ignore_list)]
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
    filtered["object_points"].tolist(),
    filtered["corners"].tolist(), shapes,
    None, None)
filtered = filtered.assign(rvecs=rvecs, tvecs=tvecs)
filtered.columns

In [ ]:
w = shapes[0]
h = shapes[1]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
(roi, newcameramtx)

In [ ]:
import logging


logger = logging.getLogger(__name__)


def draw_chess_images(row: pandas.Series):
    names = []
    error_threshold = 0.15
    axis = numpy.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)
    color = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
    undistorted_images = []

    if sample.corners is None:
        fig = plte.imshow(sample.image, title=sample.name)
        fig.show()
        logger.warning("Image %s has no detected chessboard corners, skipping...",
                       sample.name)
        return
    pre_dst = undistort_image(
        image=sample.image,
        mtx=mtx, dist=dist, new_mtx=newcameramtx,
        canvas_w=shapes[0], canvas_h=shapes[1])
    # crop the image
    x, y, w, h = roi
    dst = pre_dst # [y:y + h, x:x + w]
    imgpoints, _ = cv2.projectPoints(
        sample.obj_points, sample.rvec, sample.tvec, mtx, dist)
    error = cv2.norm(sample.corners, imgpoints, cv2.NORM_L2)/len(imgpoints)
    if error < error_threshold:
        names.append(sample.name)
    # axis2d, jac = cv2.projectPoints(axis, sample.rvec, sample.tvec, mtx, dist)
    #origin = tuple([int(v) for v in sample.corners[0].ravel()])
    #for i in range(0, 3):
    #    dst = cv2.line(dst, origin, tuple(
    #        [int(v) for v in axis2d[i].ravel()]), color[i], 5)
    lwidth = 2
    dst = cv2.line(dst, (x, y), (x + w, y), (0, 255, 0), lwidth)
    dst = cv2.line(dst, (x + w, y), (x + w, y + h), (0, 255, 0), lwidth)
    dst = cv2.line(dst, (x + w, y + h), (x, y + h), (0, 255, 0), lwidth)
    dst = cv2.line(dst, (x, y + h), (x, y), (0, 255, 0), lwidth)
    fig = plte.imshow(
        dst,
        title=f"image {sample.name},crop[x,y,w,h]={roi},error={error}")
    fig.show()

In [ ]:
# DELETE
"""
import plotly.express as plte
import requests
import logging
import dataclasses


@dataclasses.dataclass
class Grid:
    name: str
    url: str
    corners_x: int
    corners_y: int
    corners: any
    image: any
    obj_points: any = None
    rvec: numpy.ndarray = None
    tvec: numpy.ndarray = None


logger = logging.getLogger(__name__)


# set to None if all items are aceptable
acepted_items = [
    "image0_r.jpg",
    "image0_l.jpg",
    "image61_r.jpg",
    "image61_l.jpg",
    "image68_r.jpg",
    "image68_l.jpg",
    # "image75_r.jpg",
    "image75_l.jpg",
    "image83_r.jpg",
    "image83_l.jpg",
    #"image84_r.jpg",
    #"image84_l.jpg",
    # "image89_r.jpg",
    #"image89_l.jpg",
    #"image91_r.jpg",
    #"image91_l.jpg"
]

sample_dataframe = pandas.read_csv('samples.csv', sep=',')
samples: list[Grid] = []
orig_files = []
for index, item in sample_dataframe.iterrows():
    item_file = item["image"].lstrip(" ")
    url = base_url.format(item_file)
    response = requests.get(url)
    if response.status_code != 200:
        logger.error("Can't read image %s from url %s", item_file, url)
    # Draw and display the corners
    orig_image = load_image(response.content)
    orig_files.append(numpy.array(orig_image))
    if acepted_items is not None and not item_file in acepted_items:
        logger.warning("Item %s is skipped because it is not in accepted list",
                       item_file)
        continue
    corners, image = get_chess_corners_from_image(
        image=orig_image,
        corners_x=item["corners_x"],
        corners_y=item["corners_y"]
    )
    if corners is None:
        logger.error("Can't find %d, %d conrenrs in %s",
                     item["corners_x"], item["corners_y"], item_file)
    grid = Grid(
        name=item_file,
        url=url,
        corners_x=item["corners_x"],
        corners_y=item["corners_y"],
        corners=corners,
        image=image
    )
    samples.append(grid)

sample_dataframe = sample_dataframe.assign(image_data=orig_files)
"""

In [ ]:
# DELETED
"""
import logging


logger = logging.getLogger(__name__)
obj_points = []
img_points = []
objects = []
shapes = None
for sample in samples:
    if sample.corners is None:
        continue

    objp = numpy.zeros((sample.corners_x*sample.corners_y,3), numpy.float32)
    objp[:,:2] = numpy.mgrid[0:sample.corners_x,0:sample.corners_y].T.reshape(
        -1, 2)
    objp = objp
    img_shape = (sample.image.shape[1], sample.image.shape[0])
    if not shapes is None and shapes != img_shape:
        logger.error(
            "Image %s has specific shape %s that is not equals to previously shapes: %s",
            sample.name, str(img_shape), str(shapes)
            )
        continue
    if shapes is None:
        shapes = img_shape
    sample.obj_points = objp
    obj_points.append(objp)
    img_points.append(sample.corners)
    objects.append(sample)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
    obj_points, img_points, shapes, None, None)
for i in range(0, len(objects)):
    objects[i].rvec = rvecs[i]
    objects[i].tvec = tvecs[i]
ret, mtx, dist
"""

In [ ]:
"""
import logging


logger = logging.getLogger(__name__)
names = []
error_threshold = 0.15
axis = numpy.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)
color = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
undistorted_images = []

for sample in samples:
    if sample.corners is None:
        fig = plte.imshow(sample.image, title=sample.name)
        fig.show()
        logger.warning("Image %s has no detected chessboard corners, skipping...",
                       sample.name)
        continue
    pre_dst = undistort_image(
        image=sample.image,
        mtx=mtx, dist=dist, new_mtx=newcameramtx,
        canvas_w=shapes[0], canvas_h=shapes[1])
    # crop the image
    x, y, w, h = roi
    dst = pre_dst # [y:y + h, x:x + w]
    imgpoints, _ = cv2.projectPoints(
        sample.obj_points, sample.rvec, sample.tvec, mtx, dist)
    error = cv2.norm(sample.corners, imgpoints, cv2.NORM_L2)/len(imgpoints)
    if error < error_threshold:
        names.append(sample.name)
    # axis2d, jac = cv2.projectPoints(axis, sample.rvec, sample.tvec, mtx, dist)
    #origin = tuple([int(v) for v in sample.corners[0].ravel()])
    #for i in range(0, 3):
    #    dst = cv2.line(dst, origin, tuple(
    #        [int(v) for v in axis2d[i].ravel()]), color[i], 5)
    lwidth = 2
    dst = cv2.line(dst, (x, y), (x + w, y), (0, 255, 0), lwidth)
    dst = cv2.line(dst, (x + w, y), (x + w, y + h), (0, 255, 0), lwidth)
    dst = cv2.line(dst, (x + w, y + h), (x, y + h), (0, 255, 0), lwidth)
    dst = cv2.line(dst, (x, y + h), (x, y), (0, 255, 0), lwidth)
    fig = plte.imshow(
        dst,
        title=f"image {sample.name},crop[x,y,w,h]={roi},error={error}")
    fig.show()
"""

## Epipolar lines

In [ ]:
def calculate_descriptors(left_image: any, right_image: any
                          ) -> tuple[any, any]:
    sift = cv2.SIFT_create()
    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(left_image, None)
    kp2, des2 = sift.detectAndCompute(right_image, None)

    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)

    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    pts1 = []
    pts2 = []

    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.8*n.distance:
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)

    return pts1, pts2

In [ ]:
x, y, w, h = roi
first = dataframe[dataframe["no"] == 0]
first_left = undistort_image(
    image=first[first["type"] == "l"]["image_data"].iloc[0],
    mtx=mtx, dist=dist, new_mtx=newcameramtx,
    canvas_w=shapes[0], canvas_h=shapes[1])[y:y+h,x:x+w]
first_right = undistort_image(
    first[first["type"] == "r"]["image_data"].iloc[0],
    mtx=mtx, dist=dist, new_mtx=newcameramtx,
    canvas_w=shapes[0], canvas_h=shapes[1])[y:y+h,x:x+w]
left_pts, right_pts = calculate_descriptors(left_image=first_left,
                                            right_image=first_right)

In [ ]:
def draw_matched_points(
    left_image: numpy.ndarray,
    right_image: numpy.ndarray,
    left_points: numpy.ndarray,
    right_points: numpy.ndarray
) -> None:
    canvas_shape = list(left_image.shape)
    canvas_shape[1] = canvas_shape[1]*2
    canvas = numpy.zeros(canvas_shape)
    canvas[:, 0:left_image.shape[1], :] = left_image
    canvas[:, left_image.shape[1]:, :] = right_image
    for p_left, p_right in zip(left_points, right_points):
        color = tuple(numpy.random.randint(0, 255, 3).tolist())
        left_point = tuple(map(int, p_left))
        canvas = cv2.circle(canvas, left_point, 5, color,-1)
        p_right_shifted = list(p_right)
        p_right_shifted[0] = p_right_shifted[0] + left_image.shape[1]
        right_point = tuple(map(int, p_right_shifted))
        canvas = cv2.circle(canvas, right_point, 5, color,-1)
        canvas = cv2.line(canvas, left_point, right_point, (0, 255, 0), 1)
    fig = plte.imshow(canvas)
    fig.show()

In [ ]:
draw_matched_points(
    left_image=first_left,
    right_image=first_right,
    left_points=left_pts,
    right_points=right_pts
)

In [ ]:
def calculate_fundamental_matrix(
    p1: numpy.ndarray, p2: numpy.ndarray
) -> tuple[numpy.ndarray, numpy.ndarray]:
    p1 = numpy.int32(p1)
    p2 = numpy.int32(p2)
    return cv2.findFundamentalMat(p1, p2, cv2.FM_LMEDS)

In [ ]:
Fm, mask = calculate_fundamental_matrix(
    p1=left_pts, p2=right_pts
)
left_pts_filtered = numpy.array(left_pts)[mask.ravel()==1]
right_pts_filtered = numpy.array(right_pts)[mask.ravel()==1]

In [ ]:
draw_matched_points(
    left_image=first_left,
    right_image=first_right,
    left_points=left_pts_filtered,
    right_points=right_pts_filtered
)

In [ ]:
stereo = cv2.StereoBM.create(numDisparities=16, blockSize=23)
first_left_gray = cv2.cvtColor(first_left, cv2.COLOR_BGR2GRAY)
first_right_gray = cv2.cvtColor(first_right, cv2.COLOR_BGR2GRAY)
disparity = stereo.compute(first_left_gray, first_right_gray)
plte.imshow(disparity).show()

In [ ]:
!wget -O real-samples.csv https://raw.githubusercontent.com/ant-nik/semares/master/data/stereo-camera-real-test/position.csv